In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1601925593149_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import os
import sys
os.environ["PYSPARK_PYTHON"] = "/bin/python3"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_161/jre"
os.environ["SPARK_HOME"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Initialising the Spark session
spark.sparkContext.getConf().get('spark.driver.memory')

In [3]:
# check the libraries in already installed on the cluster 
spark.sparkContext.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version  
-------------------------- ---------
beautifulsoup4             4.9.1    
boto                       2.49.0   
click                      7.1.2    
jmespath                   0.10.0   
joblib                     0.16.0   
lxml                       4.5.2    
mysqlclient                1.4.2    
nltk                       3.5      
nose                       1.3.4    
numpy                      1.16.5   
pip                        9.0.1    
py-dateutil                2.2      
python37-sagemaker-pyspark 1.4.0    
pytz                       2020.1   
PyYAML                     5.3.1    
regex                      2020.7.14
setuptools                 28.8.0   
six                        1.13.0   
soupsieve                  1.9.5    
tqdm                       4.48.2   
wheel                      0.29.0   
windmill                   1.6

In [4]:
# Data stored in Public S3 Bucket 'chicago-crime-mlc' as 'Chicago_Crimes_2012_to_2017.csv'
df = spark.read.csv('s3a://advertising-dataset-ml/Advertising.csv', header = True, inferSchema = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Number of rows and columns in the dataset
print((df.count(), len(df.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(200, 5)

In [6]:
# Printing the first two row
df.head(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(_c0='1', TV='230.1', Radio='37.8', Newspaper='69.2', Sales='22.1'), Row(_c0='2', TV='44.5', Radio='39.3', Newspaper='45.1', Sales='10.4')]

In [7]:
# Schema of the dataset
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _c0: string (nullable = true)
 |-- TV: string (nullable = true)
 |-- Radio: string (nullable = true)
 |-- Newspaper: string (nullable = true)
 |-- Sales: string (nullable = true)

In [8]:
# Dropping the variables that will not contribute to the final model
df = df.drop("_c0")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
from pyspark.sql.functions import col, countDistinct
from pyspark.sql.functions import when, count, col, isnull
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Check the number of null values for each variable

In [10]:
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-----+---------+-----+
| TV|Radio|Newspaper|Sales|
+---+-----+---------+-----+
|  0|    0|        0|    0|
+---+-----+---------+-----+

In [11]:
from pyspark.ml.feature import VectorAssembler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Change the required columns from string type to numerical before passsing into the vector assembler

In [12]:
# Change the required columns from string type to numerical before applying the Vector Assembler 
from pyspark.sql.types import FloatType, IntegerType
df = df.withColumn('TV', df['TV'].cast(FloatType()))
df = df.withColumn('Radio', df['Radio'].cast(FloatType()))
df = df.withColumn('Newspaper', df['Newspaper'].cast(FloatType()))
df = df.withColumn('Sales', df['Sales'].cast(FloatType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
inputCols = [
    'TV',
    'Radio',
    'Newspaper'
]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Initialise the vector assembler as 'assembler'

In [14]:
assembler = VectorAssembler(inputCols = inputCols, outputCol = 'features')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
from pyspark.ml.regression import RandomForestRegressor

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Initialise the RandomForestRegressor as 'rf'

In [16]:
rf = RandomForestRegressor(labelCol="Sales", featuresCol="features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Splitting the dataset into train and test

In [17]:
# Splitting the dataFrame into training and testing set
train, test = df.randomSplit([0.7, 0.3], seed = 2020)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Create the pipleing and input the stages

In [18]:
from pyspark.ml import Pipeline

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
pipeline = Pipeline(stages=[assembler, rf])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Train model.  This also runs the indexer.
model = pipeline.fit(train)

# Make predictions.
predictions = model.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Select example rows to display.
predictions.select("Sales").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+
|Sales|
+-----+
|  3.2|
|  5.5|
|  7.2|
|  7.3|
|  5.3|
+-----+
only showing top 5 rows

#### Using the model created, calculate the feature importances

In [22]:
rfModel = model.stages[1]
print(rfModel)  # summary only

# Feature Importance
rfModel.featureImportances

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RandomForestRegressionModel (uid=RandomForestRegressor_ec2246050a44) with 20 trees
SparseVector(3, {0: 0.5399, 1: 0.3119, 2: 0.1482})

In [23]:
from pyspark.mllib.evaluation import RegressionMetrics

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Use RegressionMetrics to evaluate this Random Forest model

In [24]:
#important: need to cast to float type, and order by prediction, else it won't work:
preds_and_labels = predictions.select(['prediction','Sales']).withColumn('label', F.col('Sales').cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction','label'])

metrics = RegressionMetrics(preds_and_labels.rdd.map(tuple))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Squared Error
print("MSE = %s" % metrics.meanSquaredError)
print("RMSE = %s" % metrics.rootMeanSquaredError)

# R-squared
print("R-squared = %s" % metrics.r2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

MSE = 4.022514252755459
RMSE = 2.0056206652194875
R-squared = 0.8637495187281786